In [2]:
import pandas as pd
import numpy as np

In [3]:
import xgboost as xgb

In [4]:
import matplotlib.pyplot as plt
import seaborn as SNS
%matplotlib inline

In [5]:
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

In [107]:
#import daily sales data
sales = pd.read_csv('prophetdf.csv')
sales['ds']=  pd.to_datetime(sales['ds'])
#import prophet prediction
forecast = pd.read_csv('prophet_forecast.csv')
forecast = forecast[['ds', 'trend','yhat','weekly', 'yearly','holidays', 'yhat_lower', 'yhat_upper']]#'yhat_lower', 'yhat_upper'
forecast['ds']=  pd.to_datetime(forecast['ds'])
data = pd.merge(sales,forecast,on='ds',how='left')
#import weather data
weather = pd.read_csv('weather.csv')
weather['ds']=  pd.to_datetime(weather['ds'])
data = pd.merge(data,weather,on='ds',how='left')

In [108]:
#holiday_season(nov 1 - dec 23)
#data['Day'] = data['ds'].dt.day
data['Week'] = data['ds'].dt.week
data['Month'] = data['ds'].dt.month
data['Year'] = data['ds'].dt.year
data['DayofYear'] = data['ds'].dt.dayofyear
data['DayofWeek'] = data['ds'].dt.dayofweek

def holiday_season(x):
    if 305 <= x <= 357:
        return 1
    return 0
data['holiday_season'] = data['DayofYear'] .apply(holiday_season)
#remove outliers and first month to match forecast df
data = data.drop(index=[366,468,774,314,957]) #180, 213
#add offset
data['y1'] = data['y'].shift(1)
data = data[30:]
#set date to index
data = data.reset_index(drop=True)

C:\Users\Alex\AppData\Local\Temp\ipykernel_30044\2806763636.py:3: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated. Please use Series.dt.isocalendar().week instead.
  data['Week'] = data['ds'].dt.week


In [131]:
#split x / y and train / test
X = data.drop(columns = ['ds','y', 'condition', 'condition_severity', 'y1'])#'stringency_index',
y =  data['y']
X_train = X[:-7]
X_test = X[-7:]
y_train = y[:-7]
y_test = y[-7:]

In [88]:
X.head()

,trend,yhat,weekly,yearly,holidays,yhat_lower,yhat_upper,temperature,humidity,Week,Month,Year,DayofYear,DayofWeek,holiday_season,y1
0,42910.668771,30871.938406,0.038724,-0.319278,0.000000,6310.605593,54678.287934,21.1,63.0,27,7,2019,183,1,0,30217.879866
1,42941.568091,29545.852006,0.004868,-0.316820,0.000000,3603.090208,53020.134368,18.8,51.0,27,7,2019,184,2,0,29825.892188
2,42972.467412,27928.754660,-0.036591,-0.313487,0.000000,3080.075047,54489.186101,20.4,54.0,27,7,2019,185,3,0,30148.139918
3,43003.366732,25372.628485,-0.056505,-0.309357,-0.044123,2042.728777,49271.745488,22.0,51.0,27,7,2019,186,4,0,28886.698448
4,43034.266053,23177.983567,-0.143097,-0.304512,-0.013797,-1590.290740,46652.295447,25.0,56.0,27,7,2019,187,5,0,29277.998737


In [132]:
model = xgb.XGBRegressor(n_estimators=1000, colsample_bytree =0.3,
 gamma = 0.0,
 learning_rate =  0.15,
 max_depth = 5,
 min_child_weight = 1)

In [133]:
model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.3,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0.0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.15, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [134]:
y_pred = model.predict(X_test)

In [115]:
y_pred = []
for x in range(len(X_test)):
    test = X_test[x:x+1]
    #test = scaler.transform(test)
    pred = model.predict(test)
    X_test['y1'][x+len(X_train)+1] = float(pred) 
    y_pred.append(float(pred))


C:\Users\Alex\AppData\Local\Temp\ipykernel_30044\4080619257.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['y1'][x+len(X_train)+1] = float(pred)


In [135]:
print('rmse model:', np.sqrt(mean_squared_error(y_test,y_pred)))
print('rmse prophet:', np.sqrt(mean_squared_error(y_test,X_test['yhat'])))
results = pd.DataFrame()
results['actual'] = y_test
results['prediction'] = y_pred
results['prophet'] = X_test['yhat']
results

rmse model: 9712.35962470799
rmse prophet: 12383.716572385149


,actual,prediction,prophet
916,96196.124898,98141.492188,99036.496545
917,92744.263348,89269.226562,91180.400508
918,101228.700023,121530.210938,117891.200706
919,107064.467839,119446.164062,124649.179829
920,110698.874419,104985.539062,117475.785954
921,100503.117963,105892.531250,115301.625990
922,97576.039846,101735.781250,112108.702382


In [136]:
#split x / y and train / test
X = data.drop(columns = ['ds','y', 'condition', 'condition_severity', 'y1'])#'stringency_index',
y =  data['y']
X_train = X[:-23]
X_test = X[-23:-16]
y_train = y[:-23]
y_test = y[-23:-16]

In [137]:
model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.3,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0.0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.15, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [138]:
y_pred = model.predict(X_test)

In [120]:
y_pred = []
for x in range(len(X_test)):
    test = X_test[x:x+1]
    #test = scaler.transform(test)
    pred = model.predict(test)
    X_test['y1'][x+len(X_train)+1] = float(pred) 
    y_pred.append(float(pred))

C:\Users\Alex\AppData\Local\Temp\ipykernel_30044\2571079088.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['y1'][x+len(X_train)+1] = float(pred)


In [139]:
print('rmse model:', np.sqrt(mean_squared_error(y_test,y_pred)))
print('rmse prophet:', np.sqrt(mean_squared_error(y_test,X_test['yhat'])))
results = pd.DataFrame()
results['actual'] = y_test
results['prediction'] = y_pred
results['prophet'] = X_test['yhat']
results

rmse model: 16211.738124258745
rmse prophet: 16301.88670826626


,actual,prediction,prophet
900,74357.172142,83306.093750,100492.746190
901,74029.801498,68156.507812,93632.999568
902,59059.391196,55002.539062,79657.624656
903,84937.096045,59255.132812,76670.991090
904,95617.620952,77775.468750,104045.635878
905,118019.757473,94801.265625,106107.393556
906,106367.443052,92521.781250,97021.031772


In [140]:
#split x / y and train / test
X = data.drop(columns = ['ds','y', 'condition', 'condition_severity', 'y1'])#'stringency_index',
y =  data['y']
X_train = X[:-30]
X_test = X[-30:]
y_train = y[:-30]
y_test = y[-30:]

In [141]:
model.fit(X_train,y_train)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=0.3,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0.0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.15, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=5, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [142]:
y_pred = model.predict(X_test)

In [128]:
y_pred = []
for x in range(len(X_test)):
    test = X_test[x:x+1]
    #test = scaler.transform(test)
    pred = model.predict(test)
    X_test['y1'][x+len(X_train)+1] = float(pred) 
    y_pred.append(float(pred))


C:\Users\Alex\AppData\Local\Temp\ipykernel_30044\4080619257.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test['y1'][x+len(X_train)+1] = float(pred)


In [143]:
print('rmse model:', np.sqrt(mean_squared_error(y_test,y_pred)))
print('rmse prophet:', np.sqrt(mean_squared_error(y_test,X_test['yhat'])))
results = pd.DataFrame()
results['actual'] = y_test
results['prediction'] = y_pred
results['prophet'] = X_test['yhat']
results

rmse model: 27038.96730137031
rmse prophet: 18349.75476483675


,actual,prediction,prophet
893,184233.647435,177048.531250,125559.711120
894,138778.306261,138249.484375,116572.737933
895,109987.895659,122738.632812,110173.161376
896,89303.723054,105593.765625,96581.426725
897,95343.122476,131042.351562,117234.523304
898,102887.905090,143525.968750,118678.725333
899,86748.382460,112604.625000,106885.837265
900,74357.172142,107274.648438,100492.746190
901,74029.801498,99636.460938,93632.999568
902,59059.391196,83055.335938,79657.624656
